In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

In [3]:
class BERT(tf.keras.Model):
    
    def __init__(self, model_path, tokenizer_path, **kwargs):
        super(BERT, self).__init__(**kwargs)
        self.tokenizer = hub.KerasLayer(tokenizer_path)
        self.bert = hub.KerasLayer(
            model_path, trainable=False, # TF1 version could not train
            signature='tokens', signature_outputs_as_dict=True)
        self.make_type_ids = tf.keras.layers.Lambda(
            lambda x: tf.zeros(shape=tf.shape(x), dtype=tf.int32),
            name='make_type_ids')
        self.make_mask = tf.keras.layers.Lambda(
            lambda x: tf.ones(shape=tf.shape(x), dtype=tf.int32),
            name='make_mask')
    
    def call(self, input_str, segment_ids=None, input_mask=None, training=False, **kwargs):
        input_ids = self.tokenizer(input_str)
        if segment_ids is None:
            segment_ids = self.make_type_ids(input_ids)
        if input_mask is None:
            input_mask = self.make_mask(input_ids)
        output = self.bert({
            'input_ids': input_ids,
            'segment_ids': segment_ids,
            'input_mask': input_mask
        })
        return output
        # return output['pooled_output'], output['sequence_output']

In [4]:
def save_model(model_path, save_path, tokenizer_path):
    bert = BERT(model_path=model_path, tokenizer_path=tokenizer_path)
    bert._set_inputs(
        tf.keras.backend.placeholder((None, None), dtype='string'))
    bert.save(save_path, include_optimizer=False)

In [5]:
# !rm -rf ./output/*

save_model(
    model_path='./hub/chinese_L-12_H-768_A-12',
    save_path='./output/bert_zh',
    tokenizer_path='bert_token2ids')

save_model(
    model_path='./hub/chinese_wwm_ext_L-12_H-768_A-12',
    save_path='./output/bert_wwm_zh',
    tokenizer_path='bert_token2ids')

save_model(
    model_path='./hub/chinese_roberta_wwm_ext_L-12_H-768_A-12',
    save_path='./output/roberta_wwm_zh',
    tokenizer_path='bert_token2ids')

save_model(
    model_path='./hub/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16',
    save_path='./output/roberta_wwm_large_zh',
    tokenizer_path='bert_token2ids')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./output/bert_zh/assets
INFO:tensorflow:Assets written to: ./output/bert_wwm_zh/assets
INFO:tensorflow:Assets written to: ./output/roberta_wwm_zh/assets
INFO:tensorflow:Assets written to: ./output/roberta_wwm_large_zh/assets


In [7]:
for i in range(1, 12):
    save_model(
        model_path=f'./hubs/chinese_L-12_H-768_A-12-{i}',
        save_path=f'./outputs/bert_zh_{i}',
        tokenizer_path='bert_token2ids')
    save_model(
        model_path=f'./hubs/chinese_wwm_ext_L-12_H-768_A-12-{i}',
        save_path=f'./outputs/bert_wwm_zh_{i}',
        tokenizer_path='bert_token2ids')
    save_model(
        model_path=f'./hubs/chinese_roberta_wwm_ext_L-12_H-768_A-12-{i}',
        save_path=f'./outputs/roberta_wwm_zh_{i}',
        tokenizer_path='bert_token2ids')

INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_1/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_1/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_2/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_2/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_3/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_3/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_4/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_4/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_5/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_5/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_6/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_6/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_7/assets


INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_7/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_8/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_8/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_9/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_9/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_10/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_10/assets
INFO:tensorflow:Assets written to: ./outputs/bert_wwm_zh_11/assets
INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_zh_11/assets


In [9]:
for i in range(1, 24):
    save_model(
        model_path=f'./hubs/chinese_roberta_wwm_large_ext_L-24_H-1024_A-16-{i}',
        save_path=f'./outputs/roberta_wwm_large_zh_{i}',
        tokenizer_path='bert_token2ids')

INFO:tensorflow:Assets written to: ./outputs/roberta_wwm_large_zh_23/assets
